# Caso Práctico (tips) - ENGIE
*Jose Enrique Zafra Mena*

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
import json 

## Carga de los datos

In [2]:
facturas_df = pd.read_excel('datos/datos-tips/AGIkey_facturas.xlsx')
mapping_df = pd.read_excel('datos/Mapping.xlsx')
IAC_df = pd.read_excel('datos/datos-tips/MEDG2_IAC_GETRA_ABRIL_2024_06_18_13_34.xlsx')

In [3]:
facturas_df.head(1000)

,NumeroFactura,Emisor,RazonSocialEmisor,DepEmisor,Receptor,RazonSocialReceptor,DepReceptor,FechaFactura,Importe,Moneda,...,FechaRegistro,Destino,Contrato,Origen,ServicioFacturado,¿Verificada por BO?,Observaciones,Importe (<0 a pagar EEE-Enagas),mes,año
0,1400180000,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"4,56",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
1,1400180001,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"15794,20",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
2,1400180002,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"16380,50",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
3,1400180003,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"1930,22",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
4,1400180004,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"43388,45",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,1400180104,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,28/05/2024,"44751,03",NaN,...,28/05/2024,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
936,1400180105,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,28/05/2024,"11928,65",NaN,...,28/05/2024,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
937,1400180106,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,28/05/2024,"11198,08",NaN,...,28/05/2024,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
938,2024069931,ESA86484292,"ENAGAS GTS, SAU",20.0,ESB82508441,ENGIE ESPAÑA SLU,NaN,28/05/2024,"16141,36",NaN,...,28/05/2024,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN


## Filtrado de datos relevantes

### Facturas
Solo nos interesan ciertas columnas: Factura, Importe, Fecha Factura.

Además, cambiamos el formato de la fecha para poder filtrar por Abril.

In [7]:
facturas_df = facturas_df.drop_duplicates()

facturas_df = facturas_df[['NumeroFactura', 'Importe', 'FechaFactura', 'Origen','ServicioFacturado']]

# solo queremos algunos valores de Origen: C.I.Almería y Entrada PVB
facturas_filtrado_df = facturas_df[(facturas_df['Origen'].isin(['C.I. Almería'])) & (facturas_df['ServicioFacturado'].isin(['Entrada PVB']))]

# cambiamos el formato del importe
#facturas_filtrado_df['Importe'] = facturas_filtrado_df['Importe'].str.replace(',' , '.').astype(float)
facturas_filtrado_df.loc[:,'Importe'] = facturas_filtrado_df['Importe'].str.replace(',', '.').astype(float)

In [8]:
# cambiamos el formato de la fecha
facturas_filtrado_df.loc[:, 'FechaFactura'] = pd.to_datetime(facturas_filtrado_df['FechaFactura'], utc = True)

facturas_filtrado_df.head(10)

,NumeroFactura,Importe,FechaFactura,Origen,ServicioFacturado
20,2324001054,537278.06,2024-03-01 00:00:00+00:00,C.I. Almería,Entrada PVB
159,2324002755,514862.14,2024-05-02 00:00:00+00:00,C.I. Almería,Entrada PVB
376,2324004231,503763.47,2024-06-03 00:00:00+00:00,C.I. Almería,Entrada PVB
570,2324005598,536445.62,2024-04-04 00:00:00+00:00,C.I. Almería,Entrada PVB
771,2324007434,525023.53,2024-07-05 00:00:00+00:00,C.I. Almería,Entrada PVB


In [6]:

# filtramos para abril
facturas_filtrado_df = facturas_filtrado_df[facturas_filtrado_df['FechaFactura'].dt.month == 4]

#facturas_filtrado_df.head(10)

AttributeError: Can only use .dt accessor with datetimelike values

### Mapping

In [ ]:
mapping_df = mapping_df.drop_duplicates()
print(mapping_df.shape)

# solo nos interesan algunos valores: Commodity == IAC
mapping_filtrado_df = mapping_df[(mapping_df['Commodity'].isin(['IAC']))]
mapping_filtrado_df.head(200)

### Deals 
Solo queremos las columnas: Deal, Cantidad, Start-DeliveryDate, End-DeliveryDate, Procedencia (SUGST_AOC_STOK), Subflow 1, Date 1, Subcantidad 1, Subflow 2, Date 2, Subcantidad 2.

En la columna flows vemos que hay datos en json. Tratamos esa columna para sacar los datos a un formato mejor. También mantemos el Id del deal para cada subflow.

In [ ]:
IAC_df = IAC_df.drop_duplicates()

# nos quedamos con las columnas que queremos por ahora
IAC_filtrado_df = IAC_df[['Id', 'Flows', 'StartDeliveryDate', 'EndDeliveryDate']]
IAC_filtrado_df = IAC_filtrado_df.rename(columns={'Id':'Id Deal'})

IAC_filtrado_df.head()

In [ ]:
# definimos una función para tratar los json (la tendremos que usar varias veces)
def clean_json(string):
    if isinstance(string, str):
        string = string.replace("'", '"')
        string = string.replace("None", 'null')
        string = string.replace("True", 'true')
        string = string.replace("False", 'false')
    return string

# la usamos sobre la columna de flows
flows_cleaned = IAC_filtrado_df['Flows'].apply(clean_json)


# Ahora creamos otra función para extraer los datos de json a diccionarios de python
def extract_json(flows, id_deal):
    if not flows or not isinstance(flows, list):
        return pd.DataFrame()
    data = {
        'Id Deal': [],
        'Currency': [],
        'Amount': [],
        'Id': [],
        'Date': [],
        'FlowType': [],
        'PayReceive': []
    }
    for item in flows:
        if isinstance(item, dict):
            data['Id Deal'].append(id_deal)
            for key in data.keys():
                if key != 'Id Deal':
                    data[key].append(item.get(key))
        else:
            print(f"Unexpected type {type(item)}")
    return pd.DataFrame(data)

# creamos un df para meter los datos
datos_json = []

for idx, row in IAC_filtrado_df.iterrows():
    try:
        # intentamos convertir la cadena a una lista de diccionarios
        print(f"Procesamos fila {idx} con Id Deal {row['Id Deal']}")
        flows = eval(row['Flows'])
        id_deal = row['Id Deal']
        i = extract_json(flows, id_deal)
        if not i.empty:
            datos_json.append(i)
        else:
            print(f"DataFrame vacío para Id Deal {id_deal}")
    except Exception as e:
        print(e)

datos_json_final = pd.concat(datos_json, ignore_index=True)
    
# cambiamos un poco las columnas
datos_json_final = datos_json_final[['Id Deal','Id','Amount', 'Date', 'FlowType']]
datos_json_final = datos_json_final.rename(columns={'Id':'Id Flow', 'Amount':'Importe Flow', 'Date':'Fecha Flow', 'FlowType':'Subflow'})

datos_json_final.head(100)

Lo anterior solo es la columna que teníamos de flows. Ahora lo unimos con el resto del dataframe original (deals IAC). Tan solo tenemos que añadirle las columnas de las fechas iniciales y finales, según el valor de Id Deal que tenga.

También cambiamos el formato de las fechas y filtramos para el mes de Abril

In [ ]:
IAC_final_df = (pd.merge(IAC_filtrado_df, datos_json_final, on='Id Deal', how='inner')).drop_duplicates()
IAC_final_df = IAC_final_df.drop(columns=['Flows'])

# también añadimos una columna llamada Commodity con valor IAC para que sea más fácil el merge
IAC_final_df['Commodity']='IAC'

# cambiamos el formato de las fechas (por lo menos el de Fecha Flow)
IAC_final_df['Fecha Flow'] = pd.to_datetime(IAC_final_df['Fecha Flow'], utc=True)

# filtramos para el mes de Abril
IAC_final_df = IAC_final_df[IAC_final_df['Fecha Flow'].dt.month == 4]

IAC_final_df.head(100)


### Merge de Mapping con Deals
Hacemos el merge con Commodity

In [ ]:
IAC_merged_df = pd.merge(IAC_final_df, mapping_filtrado_df, on='Commodity', how='inner')

# quitamos algunas columnas innecesarias
IAC_merged_df.drop(columns=['Portfolio','DealType'], inplace=True)


IAC_merged_df.head(50)

### Merge de Mapping con Facturas
En este caso nos vale bien con Origen, o bien con Servicio Facturado.

In [ ]:
facturas_merged_df = pd.merge(facturas_filtrado_df, mapping_filtrado_df, on='Origen', how='inner')


facturas_merged_df = facturas_merged_df.rename(columns={'ServicioFacturado':'Servicio facturado'})
facturas_merged_df = facturas_merged_df.loc[:, ~facturas_merged_df.columns.duplicated()]

facturas_merged_df.head(100)

## Conciliación
**Se hace según *Origen* + *Servicio Facturado***

Mapping y Facturas comparten *Origen* y *Servicio Facturado*

Vemos la cantidad según Facturas, la cantidad según Deals, y hacemos la diferencia

### Comparación entre Facturas y Deals


In [ ]:
# agrupamos y sumamos importes
IAC_grouped = IAC_merged_df.groupby(['Id Deal','Origen', 'Servicio facturado', 'Subflow'])['Importe Flow'].sum()

IAC_grouped.head(100)

In [ ]:
# hacemos lo mismo para facturas
facturas_grouped = facturas_merged_df.groupby(['NumeroFactura','Origen', 'Servicio facturado'])['Importe'].sum()
facturas_grouped.head(100)

### DataFrame de conciliación
Falta poner columnas de Subflows, Numero de Factura, Id Deal

In [ ]:
# unimos los dataframes para comparar
conciliacion_df = pd.merge(IAC_grouped, facturas_grouped, on=['Origen', 'Servicio facturado'], how='outer')

# hallamos la diferencia 
conciliacion_df['Diferencia'] = conciliacion_df['Importe'] - conciliacion_df['Importe Flow']  

conciliacion_df.head()